In [ ]:
# https://www.data.go.kr/data/15104835/fileData.do 환승인구
# https://www.data.go.kr/data/15062858/fileData.do 환승인구
# https://www.data.go.kr/data/15044250/fileData.do 서울교통공사_승하차순위
# https://www.data.go.kr/data/15099316/fileData.do?recommendDataYn=Y 지하철 위경도
# https://www.data.go.kr/data/15097972/fileData.do 인구수
# https://www.data.go.kr/data/15126468/openapi.do 아파트 실거래가 
# https://www.data.go.kr/data/15126463/openapi.do 
# https://rt.molit.go.kr/pt/xls/xls.do?mobileAt= 상업용 실거래가
# https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=118603 행정기관코드

# https://velog.io/@kd01051/zerobase%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%B7%A8%EC%97%85%EC%8A%A4%EC%BF%A8EDA%EA%B3%BC%EC%A0%9C1%EC%8A%A4%EB%B2%85%EC%9D%B4%EB%94%94%EC%95%BC-%EC%83%81%EA%B6%8C%EB%B6%84%EC%84%9D


In [55]:
import requests
import time
from dotenv import load_dotenv
import os
import re

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


In [2]:
df_seoul = pd.read_csv('../data/raw/소상공인시장진흥공단_상가(상권)정보_서울_202503.csv', encoding='utf-8-sig')
df_subway = pd.read_csv('../data/raw/df_subway.csv', encoding='utf-8-sig')
df_subway_usr = pd.read_csv('../data/raw/서울교통공사_1_8호선 역별 일별 시간대별 승객유형별 승하차인원_20241231.csv', encoding='euc-kr')
df_population = pd.read_csv('../data/raw/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20250531.csv', encoding='euc-kr')
df_building_price = pd.read_csv('../data/raw/상업업무용(매매)_실거래가_20250623165423.csv',encoding='euc-kr',skiprows=15)

In [3]:
print(f"df_seoul: {df_seoul.shape}")
print(f"df_subway: {df_subway.shape}")
print(f"df_subway_usr: {df_subway_usr.shape}")
print(f"df_population: {df_population.shape}")
print(f"df_building_price: {df_building_price.shape}")

df_seoul: (540517, 39)
df_subway: (289, 7)
df_subway_usr: (812796, 27)
df_population: (3614, 230)
df_building_price: (10909, 21)


# 1. df_starbucks

###  매장찾기 > 지역검색
- selenium으로 클릭해 들어가서 bs4로 주소 클라스에 해당하는 값들 가져오기

In [6]:
# 스타벅스 크롤링 코드 v2.1

# 셀레니움 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스타벅스 매장찾기 페이지 접속
driver.get("https://www.starbucks.co.kr/store/store_map.do")
wait = WebDriverWait(driver, 10)


# 지역검색 탭 클릭
region_tab = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a")
))
region_tab.click()
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "set_sido_cd_btn")))

soup = BeautifulSoup(driver.page_source, "lxml")


# 서울 클릭 (XPath 사용)
# seoul_xpath = '/html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'
# seoul_btn = wait.until(EC.element_to_be_clickable((By.XPATH, seoul_xpath)))
time.sleep(2)
seoul_btn = wait.until(EC.element_to_be_clickable(
    (By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a")))
seoul_btn.click()

# 전체 클릭
# all_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "set_gugun_cd_btn")))
all_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#mCSB_2_container > ul > li:nth-child(1) > a")))
all_btn.click()




wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "quickResultLstCon")))


# 로딩 대기
time.sleep(15)
soup = BeautifulSoup(driver.page_source, "lxml")

# 매장 정보 수집
stores = soup.select("li.quickResultLstCon")
results = []
for store in stores:
    name = store['data-name']
    lat = store['data-lat']
    lng = store['data-long']
    addr = store.select_one("p.result_details").get_text(" ", strip=True)
    results.append({"매장명": name, "위도": lat, "경도": lng, "주소": addr})


driver.quit()

df_starbucks = pd.DataFrame(results)
# df.to_csv("스타벅스_서울_매장.csv", index=False)
# print(df.head())
# df_starbucks.info()


df_starbucks = df_starbucks[df_starbucks['주소'].str.contains('서울특별시')]
df_starbucks.shape

(674, 4)

In [7]:
df_starbucks['주소'] = df_starbucks['주소'].str.replace(' 1522-3232','')
df_starbucks.head()

,매장명,위도,경도,주소
0,강남구청정문,37.518181,127.045995,서울특별시 강남구 학동로 419 (청담동)
1,봉은사로선정릉,37.51131683405367,127.04846338902248,서울특별시 강남구 봉은사로 446 (삼성동)
2,선정릉역,37.51075489999999,127.04455970000003,서울특별시 강남구 봉은사로 407 (삼성동) 에프오빌딩
3,삼성현대힐,37.51350444312708,127.05273753314826,서울특별시 강남구 삼성로 605 (삼성동)
4,강남구청역,37.5165849,127.04124330000002,"서울특별시 강남구 선릉로 669, 웰빙센터 1층 (논현동)"


### (3) 행정동명, 지번주소 맵핑
- 행정동명, 행정동코드
    - 출처: 디지털 트윈국토 API (지오코더 API)
    - 함수명: get_adm_from_doro
    - 입력값(도로명주소) > 반환값(행정동)
    - 작동: 로우별로 테이블의 도로명주소를 돌면서 API에 리퀘스트 보내고 행정동, 행정동코드 받음
    - 칼럼명: 행정동, 행정동코드
    - 조회 안 되는 값 3개에 대해선 mois.go.kr에서 행정기관코드 다운
- 지번주소
    - 출처: 디지털 트윈국토 API (search API)
    - 함수명: get_jibun_from_doro
    - 입력값(도로명주소) > 반환값(지번주소)
    - 작동: 로우별로 테이블의 도로명주소를 돌면서 API에 리퀘스트 보내고 지번주소 받아냄
    - 칼럼명: '지번', '지번주소'
        - 지번: 동 + 번지
        - 지번주소: 시 + 구 + 동 + 번지

In [30]:
load_dotenv()
api_key = os.getenv("VWORLD_API_KEY")


# 스타벅스 행정동, 지번주소 변환하기 - 디지털트윈국토
# level1	문자	시·도
# level2	문자	시·군·구
# level3	문자	(일반구)구
# level4L	문자	(도로)도로명, (지번)법정읍·면·동 명
# level4LC	문자	(도로)도로코드, (지번)법정읍·면·동 코드
# level4A	문자	(도로)행정읍·면·동 명, (지번)지원안함
# level4AC	문자	(도로)행정읍·면·동 코드, (지번)지원안함
# level5	문자	(도로)길, (지번)번지


# 행정동 반환 함수
def get_adm_from_doro(address, api_key): # geocoder API
    """
    도로명주소 입력 시 행정동 반환 함수
    """
    apiurl = "https://api.vworld.kr/req/address?"	
    params = {	
        "service": "address",	
        "request": "getcoord", # getcoord: 좌표를 받겠다
        "crs": "epsg:4326",	
        # "point": point_str,	#리스트 내 스트링 형태로 받아야 함.
        "format": "json",	
        "address": address,
        "type": "road",	#입력 주소 유형(ROAD, PARCEL)
        "key": api_key	
    }	
    try:
        data = requests.get(apiurl, params=params, timeout=10)
        if data.status_code == 200:	
            # print(data.json())
            # print(data.json()['response']['error']['text']) # 디버깅
            adm = data.json()['response']['refined']['structure']
            return adm['level2'], adm['level4A'], adm['level4AC'] #구, 행정동, 행정동 코드
        else: 
            print("status != 200")
        
    except Exception as e:
        return (None, None, None)



# 지번주소 반환 함수
def get_gibun_from_doro(address, api_key): # search API
    """
    도로명주소 입력 시 지번주소 반환 함수
    """
    apiurl = "https://api.vworld.kr/req/search?"
    params = {
        "service": "search", 
        "request": "search", #필수
        "key": api_key, #필수
        "query": address, #필수
        "type": "address", #필수
        "category": "road", #필수
        "format":"json" # 기본값임
    }

    data = requests.get(apiurl, params=params).json()
    
    if data['response']['status'] == 'OK':
        return data['response']['result']['items'][0]['address']['parcel']
    else:
        print(f"오류: {data['response']['status']}")


In [9]:
df_starbucks.rename(columns={'주소':'도로명주소'}, inplace=True)

In [10]:

# 1. 행정동, 행정동코드 칼럼 생성
load_dotenv()
api_key = os.getenv("VWORLD_API_KEY")



result = df_starbucks['도로명주소'].apply(lambda x: get_adm_from_doro(x, api_key))
gu, dong, adm_cd = zip(*result)

df_starbucks['행정동명'] = dong # 3개 누락값있음
df_starbucks['행정동코드'] = adm_cd # 3개 누락값있음


In [12]:
null_df = df_starbucks[df_starbucks['행정동명'].isna()]
null_df

,매장명,위도,경도,도로명주소,행정동명,행정동코드
204,가산디지털,37.479970085901,126.882507193785,"서울특별시 금천구 가산디지털1로 168 (가산동), 우림라이온스밸리 B동 1층",None,None
212,월계역,37.6295,127.0574,"서울특별시 노원구 초안산로2라길26 월계동(104,105,106호)",None,None
258,서강대흥역,37.548178,126.941576,서울특별시 마포구 백범로 89-5 (대흥동),None,None
425,목동SBS,37.52896162444895,126.87363009594163,"서울특별시 양천구 목동서로 161 (목동), 101호, 201호",None,None
544,서울중앙우체국,37.561678,126.982117,서울특별시 중구 소공로 70 (충무로 1가) 서울 중앙 우체국,None,None
566,명동역,37.5608622201549,126.983105390255,"서울특별시 중구 퇴계로 101, 건물 전체 (충무로 1가)",None,None


In [14]:
null_df['도로명주소'] = null_df['도로명주소'].str.split('(').str[0]

/var/folders/q2/0w8s3xgj6szdvf0r9_0lyqfr0000gn/T/ipykernel_70716/258106746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df['도로명주소'] = null_df['도로명주소'].str.split('(').str[0]


In [18]:
new_data = null_df['도로명주소'].apply(lambda x: get_adm_from_doro(x, api_key))
df_starbucks.loc[null_df.index, '행정동명'] = new_data.apply(lambda x: x[1])
df_starbucks.loc[null_df.index, '행정동코드'] = new_data.apply(lambda x: x[2])

df_starbucks[df_starbucks['행정동명'].isna()]

,매장명,위도,경도,도로명주소,행정동명,행정동코드
258,서강대흥역,37.548178,126.941576,서울특별시 마포구 백범로 89-5 (대흥동),None,None
425,목동SBS,37.52896162444895,126.87363009594163,"서울특별시 양천구 목동서로 161 (목동), 101호, 201호",None,None


In [25]:
df_starbucks.loc[df_starbucks['매장명'] == '서강대흥역', '행정동명'] = '대흥동' # 1144060000
df_starbucks.loc[df_starbucks['매장명'] == '목동SBS', '행정동명'] = '목1동' # 1147051000

df_starbucks.loc[df_starbucks['매장명'] == '서강대흥역', '행정동코드'] = 1144060000
df_starbucks.loc[df_starbucks['매장명'] == '목동SBS', '행정동코드'] = 1147051000
df_starbucks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     674 non-null    object
 1   위도      674 non-null    object
 2   경도      674 non-null    object
 3   도로명주소   674 non-null    object
 4   행정동명    674 non-null    object
 5   행정동코드   674 non-null    object
dtypes: object(6)
memory usage: 31.7+ KB


In [26]:
print(df_starbucks.shape)
df_starbucks = df_starbucks.drop_duplicates()
print(df_starbucks.shape)

(674, 6)
(644, 6)


In [31]:
# 지번주소 생성하기
df_starbucks['지번주소'] = df_starbucks['도로명주소'].apply(lambda x: get_gibun_from_doro(x, api_key))
df_starbucks['지번주소'].isna().sum()

오류: NOT_FOUND
오류: NOT_FOUND


np.int64(2)

In [32]:
df_starbucks[df_starbucks['지번주소'].isna()]

,매장명,위도,경도,도로명주소,행정동명,행정동코드,지번주소
258,서강대흥역,37.548178,126.941576,서울특별시 마포구 백범로 89-5 (대흥동),대흥동,1103071000,None
425,목동SBS,37.52896162444895,126.87363009594163,"서울특별시 양천구 목동서로 161 (목동), 101호, 201호",목1동,1115051000,None


In [33]:
df_starbucks.loc[df_starbucks['매장명'] == '서강대흥역', '지번주소'] = '대흥동 119 89-5'
df_starbucks.loc[df_starbucks['매장명'] == '목동SBS', '지번주소'] = '목동 920'

In [46]:
from joblib.externals.loky.backend.context import _DEFAULT_START_METHOD


pattern = r'서울특별시\s+[가-힣]+구'

gu = df_starbucks['도로명주소'].apply(lambda x: re.search(pattern, x).group(0))
gibbun = df_starbucks['지번주소']

df_starbucks['지번주소'] = gu + " " +gibbun

df_starbucks.head(1)


,매장명,위도,경도,도로명주소,행정동명,행정동코드,지번주소
0,강남구청정문,37.518181,127.045995,서울특별시 강남구 학동로 419 (청담동),청담동,1168056500,서울특별시 강남구 청담동 44-12


### (4) df_seoul과 정합성을 위한 컬럼 생성

In [48]:
# df_seoul에서 사용할 컬럼 목록:
# '상가업소번호', '상호명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '시도명', '시군구명',
# '행정동코드', '행정동명', '법정동명', '지번주소', '도로명주소', '경도', '위도', 
df_starbucks['스타벅스'] = 1
df_starbucks['상권업종대분류명'] = '음식'
df_starbucks['상권업종중분류명'] = '비알코올'
df_starbucks['상권업종소분류명'] = '카페'
df_starbucks['상가업소번호'] = 'sb' + df_starbucks.index.astype(str)

df_starbucks = df_starbucks.rename(columns={'매장명':'상호명'})
len(df_starbucks.columns) # 12

12

In [51]:
os.makedirs('../processed', exist_ok=True)
df_starbucks.to_csv('../processed/df_starbucks.csv', encoding='utf-8-sig', index=False)

# 2. df_seoul

데이터 출처: https://www.data.go.kr/data/15083033/fileData.do

In [52]:
df_seoul = df_seoul[['상가업소번호', '상호명', 
    #  '지점명', '상권업종대분류코드', 
        '상권업종대분류명', 
    #  '상권업종중분류코드',
       '상권업종중분류명', 
    #    '상권업종소분류코드', 
       '상권업종소분류명', 
    #    '표준산업분류코드', '표준산업분류명', '시도코드',
      #  '시도명', 
    #    '시군구코드', 
       '시군구명', '행정동코드', '행정동명', 
    #    '법정동코드', 
      #  '법정동명', 
    #    '지번코드','대지구분코드', '대지구분명', '지번본번지', '지번부번지', 
       '지번주소',
    # '도로명코드', '도로명', '건물본번지','건물부번지', '건물관리번호', '건물명',
       '도로명주소', 
    #  '구우편번호', '신우편번호', '동정보', '층정보','호정보', 
       '경도', '위도']]
df_seoul['스타벅스'] = 0
df_seoul.to_csv('../processed/df_seoul.csv', encoding='utf-8-sig', index=False)

# 3. df_population

데이터 출처: 
- (1) 행정동 인구: https://www.data.go.kr/data/15097972/fileData.do
- (2) 행정동 위경도: https://skyseven73.tistory.com/23, 누락 데이터는 수기 검색 후 보강.

- 행정동별 성별 연령별 주민등록 인구수 집계
- 나이 구간 설정:
    - ~6 어린이
    -  7~19 학생
    - 20~25 대학생
    - 26~30 초년생
    - 31~40 신혼부부 등
    - 40~50 자녀를 둔 부부 등
    - 50~60 중년
    - 60~ 노년
- 함수: make_bins


### (1) 나이 구간 설정

In [62]:
df_population = pd.read_csv('../raw/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20250531.csv', encoding='euc-kr')

# 0세부터 나이가 모두 나와있음. 정제 필요
df_frame = df_population.iloc[:,:6]


col_male = [col for col in df_population.columns.to_list() if col in re.findall(r'.*남자.*', col)]
df_male = df_population[col_male]
col_male = [col.replace('남자','') for col in col_male]
df_male.columns = col_male

col_female = [col for col in df_population.columns.to_list() if col in re.findall(r'.*여자.*', col)]
df_female = df_population[col_female]
col_female = [col.replace('여자','') for col in col_female]
df_female.columns = col_female

df_total = df_male + df_female
df_total = pd.concat([df_frame, df_total], axis=1)

# 나이는 어떻게 나누면 좋을까?
## ~6 어린이
## 7~19 학생
## 20~25 대학생
## 26~30 초년생
## 31~40 신혼부부 등
## 40~50 자녀를 둔 부부 등
## 50~60 중년
## 60~ 노년

def make_bins(df):
    r = [(0,6), (7,19), (20,25), (26,30), (31,40), (41,50), (51,60), (61,109)]
    new_cols = []
    for start_age, end_age in r:

        start_col = f'{start_age}세'
        start_loc = df.columns.get_loc(start_col)

        end_col = f'{end_age}세'
        end_loc = df.columns.get_loc(end_col)
        new_col_name = f'{start_age}세_{end_age}세'
        df[new_col_name] = df.iloc[:, start_loc:end_loc+1].sum(axis=1)
        new_cols.append(new_col_name)
    return df, new_cols

df_total, pop_cols = make_bins(df_total)

df_population = pd.concat([df_frame, df_total[pop_cols]], axis=1)
df_population.drop(columns=['기준연월'], inplace=True)
df_population.head()


,행정기관코드,시도명,시군구명,읍면동명,계,0세_6세,7세_19세,20세_25세,26세_30세,31세_40세,41세_50세,51세_60세,61세_109세
0,1111051500,서울특별시,종로구,청운효자동,10978,309,1239,680,797,1486,1775,1849,2843
1,1111053000,서울특별시,종로구,사직동,8906,225,782,501,681,1376,1396,1457,2488
2,1111054000,서울특별시,종로구,삼청동,2157,41,189,108,143,275,298,333,770
3,1111055000,서울특별시,종로구,부암동,9030,188,940,565,592,1176,1383,1604,2582
4,1111056000,서울특별시,종로구,평창동,17210,554,1871,1055,1050,2034,2427,3094,5125


In [63]:
df_population.to_csv('../processed/df_population.csv', encoding='utf-8-sig', index=False)

# 4.df_building_price

### (1) 컬럼 생성: 주소
데이처 출처: https://rt.molit.go.kr/pt/xls/xls.do?mobileAt=
- 법정동 기준이라 행정동명과 차이가 있음
- 칼럼 생성: 
    - 주소: 시군구 + 지번
### (2) 컬럼 생성: 좌표
- 컬람 생성:
    - x: 경도
    - y: 위도
    - 함수: get_coords_vworld
        - 지번주소 입력 시 좌표계 반환 
    - 디지털트윈국토 API 활용

In [66]:
df_building_price = pd.read_csv('../raw/상업업무용(매매)_실거래가_20250623165423.csv',encoding='euc-kr',skiprows=15)
df_building_price.head(1).T

,0
NO,1
시군구,서울특별시 동작구 노량진동
유형,집합
지번,283-3
도로명,노량진로6길
용도지역,제3종일반주거
건축물주용도,제2종근린생활
도로조건,12m미만
전용/연면적(㎡),9.88
대지면적(㎡),


In [72]:
# (1) 칼럼 생성: 주소
df_building_price = df_building_price[['시군구','지번','도로명','용도지역','건축물주용도','전용/연면적(㎡)','거래금액(만원)']]
df_building_price['주소'] = df_building_price['시군구'] + ' ' + df_building_price['지번']
df_building_price['도로명'].isna().sum() # 284


np.int64(284)

In [73]:
df_building_price.head(1)

,시군구,지번,도로명,용도지역,건축물주용도,전용/연면적(㎡),거래금액(만원),주소
0,서울특별시 동작구 노량진동,283-3,노량진로6길,제3종일반주거,제2종근린생활,9.88,"7,300",서울특별시 동작구 노량진동 283-3


In [ ]:
# (2) 칼럼 생성: 좌표
# 좌표 반환 함수
def get_coords_vworld(address, api_key):
    """지번주소 입력시 좌표 반환"""
    url = "https://api.vworld.kr/req/address"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "PARCEL", #지번주소
        "key": api_key
    }
    try:
        res = requests.get(url, params=params, timeout=0.5)
        if res.status_code != 200:
            return None, None
        data = res.json()
        if data.get('response', {}).get('status') == 'OK':
            data = res.json()['response']['result']['point']
            x = data['x']
            y = data['y']
            return x, y
    except Exception as e:
        print(f"오류 발생: {e} (주소: {address})")
    return None, None


lat_lon = df_building_price['주소'].apply(lambda x: get_coords_vworld(x, api_key))

lon, lat = zip(*lat_lon)
# lon, lat = lat_lon.apply(pd.Series)

df_building_price['경도'] = lon
df_building_price['위도'] = lat


오류 발생: HTTPSConnectionPool(host='api.vworld.kr', port=443): Read timed out. (read timeout=0.5) (주소: 서울특별시 용산구 원효로1가 4*)
오류 발생: HTTPSConnectionPool(host='api.vworld.kr', port=443): Read timed out. (read timeout=0.5) (주소: 서울특별시 강남구 역삼동 755)
오류 발생: HTTPSConnectionPool(host='api.vworld.kr', port=443): Read timed out. (read timeout=0.5) (주소: 서울특별시 강서구 화곡동 105-102)


In [77]:

print(df_building_price.head())
print(df_building_price.shape)
df_building_price = df_building_price.dropna(axis=0)

               시군구      지번     도로명     용도지역   건축물주용도  전용/연면적(㎡) 거래금액(만원)  \
0   서울특별시 동작구 노량진동   283-3  노량진로6길  제3종일반주거  제2종근린생활       9.88    7,300   
1    서울특별시 양천구 신정동   911-4   신정중앙로  제2종일반주거  제1종근린생활      96.39    7,000   
2    서울특별시 성동구 행당동  286-37  무학봉26길      준주거  제2종근린생활      24.57   17,500   
3     서울특별시 중구 신당동     773     마장로     일반상업       업무      37.73   28,000   
4  서울특별시 동대문구 청량리동     738     홍릉로     일반상업       판매       6.46    2,750   

                     주소                  경도                  위도  
0  서울특별시 동작구 노량진동 283-3  126.93601158189202   37.51084899878593  
1   서울특별시 양천구 신정동 911-4  126.85800748073522  37.526895534764535  
2  서울특별시 성동구 행당동 286-37  127.03388083120328  37.561164591843365  
3      서울특별시 중구 신당동 773  127.01060051259533   37.56882662555164  
4   서울특별시 동대문구 청량리동 738  127.04439603663937  37.581418245465976  
(9906, 10)


In [78]:
df_building_price.to_csv('../processed/df_building_price.csv', encoding='utf-8-sig', index=False) 
df_building_price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9906 entries, 0 to 10908
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   시군구        9906 non-null   object 
 1   지번         9906 non-null   object 
 2   도로명        9906 non-null   object 
 3   용도지역       9906 non-null   object 
 4   건축물주용도     9906 non-null   object 
 5   전용/연면적(㎡)  9906 non-null   float64
 6   거래금액(만원)   9906 non-null   object 
 7   주소         9906 non-null   object 
 8   경도         9906 non-null   object 
 9   위도         9906 non-null   object 
dtypes: float64(1), object(9)
memory usage: 851.3+ KB


# 5.df_subway, df_subway_on , df_subway_off

- 참고: 횡단면 데이터로 2024-07-01 시점을 기준으로 하기 떄문에 시계열 반영 필요.
- 데이터 출처: 
    - https://www.data.go.kr/data/15099316/fileData.do?recommendDataYn=Y 역사 좌표 값
    - https://www.data.go.kr/data/15104835/fileData.do 환승인구
    - https://www.data.go.kr/data/15062858/fileData.do 환승인구
    - https://www.data.go.kr/data/15044250/fileData.do 서울교통공사_승하차순위
- df_subway: 지하철 역사 좌표값
- df_subway_on, df_subway_off
    - df_subway_usr1: 1호선 ~ 8호선 승하차 데이터
    - df_subway_usr2: 9호선 승하차 데이터
    - (1) 데이터 concat: df_subway_usr1, df_subway_usr2
    - (2) 칼럼 생성: 출근 시간, 퇴근시간 승하차 수
        - 출근 시간: 07시 ~ 10시 
        - 퇴근 시간: 17시 ~ 20시
    - (3) 데이터 프레임 분리: 승하차로 구분


In [79]:
# 1호선 ~ 8 호선 시간대별 이용객수
# 승객유형 칼럼 제외
df_subway_usr1 = pd.read_csv('../raw/서울교통공사_1_8호선 역별 일별 시간대별 승객유형별 승하차인원_20241231.csv', encoding='euc-kr').dropna(axis=0)
df_subway_usr1['이용객수'] = df_subway_usr1.iloc[:,7:].sum(axis=1)
df_subway_usr1 = df_subway_usr1[['수송일자','호선명','역번호','역명','승하차구분','이용객수','07-08시간대','08-09시간대','09-10시간대','17-18시간대','18-19시간대','19-20시간대']]

# 9호선 시간대별 이용객수
## 1호선~8호선 데이터 프레임 기준으로 칼럼명 맞춰줌.
df_subway_usr2 = pd.read_csv('../raw/서울교통공사_9호선2_3단계 역별일별시간대별승하차인원_20241031.csv', encoding='euc-kr')
for i in range(12,30):
    df_subway_usr2.iloc[:,i] = pd.to_numeric(df_subway_usr2.iloc[:,i].str.replace(',',''), errors='coerce')
df_subway_usr2['이용객수'] = df_subway_usr2.iloc[:,6:].sum(axis=1)

df_subway_usr2 = df_subway_usr2[['날짜','호선','역번호','역사명','구분','이용객수','07시-08시','08시-09시','09시-10시','17시-18시','18시-19시','19시-20시']]

df_subway_usr2.loc[df_subway_usr2['구분'] == '순승차','구분'] = '승차'
df_subway_usr2.loc[df_subway_usr2['구분'] == '순하차','구분'] = '하차'
df_subway_usr2.columns = df_subway_usr1.columns

# (1) 데이터 콘캣: 1호선-8호선 데이터 + 9호선 데이터
df_subway_usr = pd.concat([df_subway_usr1, df_subway_usr2], axis=0)
df_subway_usr = df_subway_usr.round()

# (2) 날짜별 합계를 구한다음 역사별 평균값을 구함. 
df_subway_usr = df_subway_usr.groupby(['수송일자','호선명','역번호','역명','승하차구분'])[['이용객수', '07-08시간대','08-09시간대','09-10시간대','17-18시간대','18-19시간대','19-20시간대']].sum().reset_index()
df_subway_usr = df_subway_usr.groupby(['호선명','역번호','역명','승하차구분'])[['이용객수', '07-08시간대','08-09시간대','09-10시간대','17-18시간대','18-19시간대','19-20시간대']].mean().reset_index()

# (3) 시간대 합산: 출근시간대, 퇴근시간대
df_subway_usr['출근시간대'] = df_subway_usr['07-08시간대'] + df_subway_usr['08-09시간대'] + df_subway_usr['09-10시간대']
df_subway_usr['퇴근시간대'] = df_subway_usr['17-18시간대'] + df_subway_usr['18-19시간대'] + df_subway_usr['19-20시간대']
df_subway_usr = df_subway_usr[['역명','역번호','호선명','승하차구분','이용객수','출근시간대','퇴근시간대']]
# df_subway_usr.to_csv('df_subway_usr.csv', encoding='utf-8-sig')

# (3) df 분리: 승차, 하차
df_subway_on = df_subway_usr[df_subway_usr['승하차구분']=='승차'].drop(columns='승하차구분',axis=1)
df_subway_off = df_subway_usr[df_subway_usr['승하차구분']=='하차'].drop(columns='승하차구분',axis=1)

df_subway_on.columns = ['역명','역번호','호선명','승차_이용객수','승차_출근시간대','승차_퇴근시간대']
df_subway_off.columns = ['역명','역번호','호선명','하차_이용객수','하차_출근시간대','하차_퇴근시간대']


In [80]:
# "역명"을 기준으로 sum : 1호선 서울역과 4호선 서울역 모두 서울역으로 취급 예정. (e.g. 스타벅스에서 가장 가까운 역: 서울역)
df_subway_on[df_subway_on['역명'].str.contains(r'.*역$')]

,역명,역번호,호선명,승차_이용객수,승차_출근시간대,승차_퇴근시간대
26,서울역,150,1호선,54331.054348,5986.918478,17562.353261
246,서울역,426,4호선,12373.130435,1469.48913,4611.211957


In [81]:

df_subway_on.to_csv('../processed/df_subway_on.csv', encoding='utf-8-sig', index=False)
df_subway_off.to_csv('../processed/df_subway_off.csv', encoding='utf-8-sig', index=False)

# 6. df_subway

In [14]:
df_subway = pd.read_csv('subway.csv', encoding='utf-8-sig')
df_subway = df_subway.iloc[:,1:]
df_subway.head()

,연번,호선,역번호,역명,위도,경도
0,1,9,926,언주역,37.507324,127.033905
1,2,9,927,선정릉역,37.510278,127.043902
2,3,9,928,삼성중앙역,37.513060,127.053334
3,4,9,929,봉은사역,37.514258,127.060257
4,5,9,930,종합운동장역,37.511433,127.076313


In [ ]:
mega_df = df[df['상호명'].str.contains('메가엠지씨커피', na=False)]
mega_df.to_csv('mega_df.csv', encoding='utf-8-sig')